# Modelo de detección positividad

## Descargas, instalaciones e importaciones

In [9]:
# Importaciones necesarias

# Para dataframe
import pandas as pd
# Para limpieza de texto
import nltk
from nltk.corpus.reader.tagged import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.text import FreqDist
# Para guardar resultados en archivos
import pickle
# Para vectorizar el texto
from sklearn.feature_extraction.text import TfidfVectorizer
# Para preparar los datos vectorizados
from sklearn.model_selection import train_test_split

In [10]:
# Metricas para clasificación
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
# Descargas de nltk necesarias

# Signos de puntuación
nltk.download('punkt')
# Stopwrds
nltk.download("stopwords")
# Nombres propios
nltk.download("names")
# Lemmatizer
nltk.download('wordnet')


In [12]:
# Declarando las listas de cosas que limpiar en el texto
stopwords = nltk.corpus.stopwords.words('english')
nombres = nltk.corpus.names.words()

## Unboxing y preparación de la data

In [13]:
# Creando nuestro dataframe
df = pd.read_csv("Text_Emotion.csv")

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
# Ver la estructura del dataframe
df.head(3)

In [ ]:
df["emotion"].value_counts()

In [18]:
# Cambiando las etiquetas a texto

df['emotion'] = df['emotion'].replace('☹️', "negative")
df['emotion'] = df['emotion'].replace('🙂', "positive")


In [ ]:
df.head(3)

## Trtamiento de la texto para su uso

In [20]:
# Instanciando el lematizador
lematizador = WordNetLemmatizer()

In [21]:
# Palabras indeseadas
palabras_indeseadas = ['he', 'a', 'the', 'in', 'an', 'it', 'she', 'ca', 'wo']

In [22]:
# Función para tokenizar y limpiar el texto
def obtener_tokens(text: str) -> list:

  """
  Función para tokenizar y limpiar el texto.
  Se obvian los tokens que sean stopwords,
  nombres, números, caracteres especiales,
  letras palabras indeseadas. Los demás tokens
  se lematizan y son recolectados.
  """

  tokens_crudos = word_tokenize(text)
  tokens = []
  for token in tokens_crudos:
    if token in stopwords: continue
    if token in nombres: continue
    if not token.isalpha():  continue
    if (len(token) < 2): continue
    token = token.lower()
    if token in palabras_indeseadas: continue
    token = lematizador.lemmatize(token)
    tokens.append(token)

  return tokens



In [23]:
# Crear los vocabularios
vocabulario = {emotion:[] for emotion in df['emotion'].unique()}
vocabulario_completo = []
for _, row in df.iterrows():
  tokens = obtener_tokens(row["text"])
  vocabulario[row['emotion']] += tokens
  vocabulario_completo += tokens

In [ ]:
# Ver los tokens mas comunes en vocabulario_completo

# Crear un objeto FreqDist para contar las frecuencias de las palabras
freq_dist = FreqDist(vocabulario_completo)

# Obtener las palabras más comunes
palabras_mas_comunes = freq_dist.most_common(5)

# Imprimir las palabras más comunes
print("Palabras más comunes en vocabulario_completo:")
for palabra, frecuencia in palabras_mas_comunes:
  print(f"{palabra}: {frecuencia}")


In [ ]:
# Contar las ocurrencias de cada token por emoción
vocabulario_por_emocion = {emotion: FreqDist(tokens) for emotion, tokens in vocabulario.items()}

# Obtener los tokens más comunes por emoción
tokens_mas_comunes_por_emocion = {emotion: freq_dist.most_common(5) for emotion, freq_dist in vocabulario_por_emocion.items()}

# Imprimir los tokens más comunes por emoción
for emotion, tokens in tokens_mas_comunes_por_emocion.items():
  print(f"Tokens más comunes para la emoción '{emotion}':")
  for token, count in tokens:
    print(f"\t{token}: {count}")

In [26]:
# Guardar los tokens más comunes en total
tokens_mas_comunes = []
for emotion in list(df['emotion'].unique()):
  tokens_mas_comunes += list([i[0] for i in FreqDist(vocabulario[emotion]).most_common(10000)])

In [ ]:
tokens_mas_comunes = set(tokens_mas_comunes)
len(tokens_mas_comunes)

In [48]:
# Se guardan los tokens más comunes
with open("most_common_tokens_positivity.pkl", "wb") as file:
  pickle.dump(tokens_mas_comunes, file)

In [29]:
# Funcion para limpiar y asignar los tokens en el dataset
def obtener_tokens_de_entrenamiento(text: str) -> str:

  """
  Función para tokenizar y limpiar el texto.
  Se obvian los tokens que sean stopwords,
  nombres, números, caracteres especiales,
  letras y palabras indeseadas. Los demás tokens
  se lematizan y son devueltos unidos en un string
  """

  tokens_crudos = word_tokenize(text)
  tokens = []
  for token in tokens_crudos:
    if token in stopwords: continue
    if token in nombres: continue
    if not token.isalpha():  continue
    if (len(token) < 2): continue
    token = token.lower()
    if token in palabras_indeseadas: continue
    if token not in tokens_mas_comunes: continue
    token = lematizador.lemmatize(token)
    tokens.append(token)

  return ' '.join(tokens)

In [30]:
# Agregamos los tokens correspondiente a cada review
df['Tokens'] = df["text"].apply(lambda x: obtener_tokens_de_entrenamiento(x))

In [ ]:
df.head()

In [32]:
# Creación de data frame de entrenamiento
training_df = pd.concat([df[df["emotion"] == emotion].sample(60000, replace=True, random_state=1) for emotion in df["emotion"].unique()])

In [ ]:
len(training_df)

In [ ]:
training_df.sample(10)

In [35]:
# Vectorizamos la data para entrarla al modelo
vectorizer = TfidfVectorizer(vocabulary=tokens_mas_comunes)
x = vectorizer.fit_transform(training_df["Tokens"])

In [49]:
with open("vectorizador_positivity.pkl", "wb") as file:
  pickle.dump(x, file)

In [37]:
# Asignamos las etiquetas
y = training_df["emotion"]

In [38]:
# Dividimos la data que utilizaremos
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.35, random_state=0)

In [39]:
from sklearn.ensemble import RandomForestClassifier

# Modelo más efectivo
random_forest_model = RandomForestClassifier(random_state=0)

# Lista de modelos
models_list = [random_forest_model]

In [40]:
def entrenamiento_de_modelos(lista_de_modelos: list, X_train, y_train, X_test, y_test):

  """
  Entrena la lista de modelos que le pasen
  con la data vectorizada con tfidf y se
  recolectan las métricas
  """
  
  lista_de_resultados = []

  for modelo in lista_de_modelos:
    # Entrenamiento
    modelo.fit(X_train.toarray(), y_train)
    y_prediccion = modelo.predict(X_test.toarray())

    # Métricas
    report = classification_report(y_prediccion, y_test)
    matrix = confusion_matrix(y_prediccion, y_test)
    accuracy = accuracy_score(y_prediccion, y_test)

    # Añadiendo los reusltados
    lista_de_resultados.append((str(modelo), report, matrix, accuracy))

  return lista_de_resultados


In [41]:
# Se entrenan los modelos y se obtienen las analíticas
resultados_de_modelos = entrenamiento_de_modelos(models_list, X_train, y_train, X_test, y_test)

In [ ]:
# Analisis de los modelos
for resultado in resultados_de_modelos:
  print(f"""
  {resultado[0]}

  {resultado[1]}

  {resultado[2]}

  {resultado[3]}

  ------------------------------------------------------------ """)

In [50]:
final_positivity_model = random_forest_model.fit(X_train.toarray(), y_train)

In [51]:
# Guardamos el mejor modelo
with open("positivity_model.pkl", "wb") as file:
  pickle.dump(final_positivity_model, file)